<a href="https://colab.research.google.com/github/nsp8/HexGridGenerator/blob/master/SimpleHexGridGenerator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HexGrid generator of a set of countries on the world map
### Using libraries like: GeoPandas and GeoJSON

In [0]:
!pip install geojson geopandas

In [0]:
from google.colab import drive
import geojson
import geopandas as gpd
import json
import math
import os
import pandas as pd

In [0]:
def get_regular_hex_coords(x, y, r):
#     r = math.sqrt(x**2 + y**2)
#     print("country: {}\tradius: {}".format(c, r))
    coords_ = list()
    n = 6
    base_angle = 360 // n
    for i in range(n):
        relative_angle = ((base_angle * i) + (base_angle // 2)) * (math.pi / 180)
        coord_ = [x + (r * math.cos(relative_angle)), y + (r * math.sin(relative_angle))]
        coords_.append(coord_)
    coords_.append(coords_[0])
    return coords_

In [0]:
input_directory = '/gdrive/My Drive/input_folder/'
output_directory = '/gdrive/My Drive/output_folder/'

### After this cell executes, it will prompt you to authorize your Google account.
- Click on the link as described below
- Select a Google account for authorization
- Copy the code and paste it in the space below and you are all set!

In [0]:
hexcode_file = os.path.join(input_directory, 'hexcodes_expanded.csv')
drive.mount('/gdrive')
with open(hexcode_file, 'r') as hexcode_maps:
    hexcodes = pd.read_csv(hexcode_maps, encoding="UTF-8")

In [0]:
countries = hexcodes["id"]
country_param = lambda c, i: hexcodes[countries == c][i].squeeze()

In [0]:
world_hex = geojson.FeatureCollection(features=[])
i = 0
radius = 1 / math.sin(math.pi/3)
for country in list(countries):
#     print("country = {}".format(country))
    params = {"label": country,
              "x": int(country_param(country, "x")),
              "y": int(country_param(country, "y"))
             }
    hex_coords = get_regular_hex_coords(params["x"], params["y"], radius)
    country_hexagon = geojson.Polygon([hex_coords])
    country_feature = geojson.Feature(geometry=country_hexagon, 
                                      properties=params)
    world_hex["features"].append(country_feature)
    i += 1


In [0]:
out_file = os.path.join(output_directory, 'new_output.geojson')
with open(out_file, "w") as output:
    json.dump(world_hex, output, indent=4)

### This GeoJSON file can be visualized in Flourish
Something like this: https://app.flourish.studio/visualisation/636299